<a href="https://colab.research.google.com/github/R4HUL-ROY/Machine-Learning/blob/main/decision_tree_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Assignment on Decision Tree Classification**

**NAME : RAHUL ROY**

**MCA 2nd Sem**

**ROLL - C91/MCA/202032**

In [1]:
import numpy as np
import pandas as pd
import math

**CHOOSE AND READ DATASET**

In [ ]:
# DATASET - 1

data_dict = {
    "age" : ['youth', 'youth','middle-aged','senior','senior','senior','middle-aged','youth','youth','senior','youth','middle-aged','middle-aged','senior'],
    "income" : ['high','high','high','medium','low','low','low','medium','low','medium','medium','medium','high','medium'],
    "student" : ['no','no','no','no','yes','yes','yes','no','yes','yes','yes','no','yes','no'],
    "credit_rating" : ['fair','excellent','fair','fair','fair','excellent','excellent','fair','fair','fair','excellent','excellent','fair','excellent'], 
    "buys" : ['no','no','yes','yes','yes','no','yes','no','yes','yes','yes','yes','yes','no']
}

# x_train_y_train = pd.DataFrame(data_dict)
entire_df = pd.DataFrame(data_dict)

In [ ]:
## DATASET - 2
# CAR DATASET    link : http://archive.ics.uci.edu/ml/datasets/Car+Evaluation

entire_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/car.csv')

## renaming the column names
col_name = ['buying', 'maint', 'doors' , 'persons', 'lug_boot', 'safety', 'class']
entire_df.columns = col_name

print(entire_df.shape)


(1727, 7)


In [4]:
# DATASET 3
# Nursery Dataset,    link for dataset  https://archive.ics.uci.edu/ml/datasets/Nursery

entire_df = pd.read_csv('/content/drive/MyDrive/Machine Learning/ML DATASET/nursery.csv')

col_name = ['parents', 'has_nurs', 'form', 'children', 'housing', 'finance', 'social', 'health', 'class']
entire_df.columns = col_name

print(entire_df.shape)

(12959, 9)


In [5]:
entire_df.head()

,parents,has_nurs,form,children,housing,finance,social,health,class
0,usual,proper,complete,1,convenient,convenient,nonprob,priority,priority
1,usual,proper,complete,1,convenient,convenient,nonprob,not_recom,not_recom
2,usual,proper,complete,1,convenient,convenient,slightly_prob,recommended,recommend
3,usual,proper,complete,1,convenient,convenient,slightly_prob,priority,priority
4,usual,proper,complete,1,convenient,convenient,slightly_prob,not_recom,not_recom


**SPLITTING THE DATASET INTO X_TRAIN_Y_TRAIN AND X_TEST AND Y_TEST**

In [6]:
## SPLITTING DATASET INTO 80 - 20 RATIO##

def split_data(entire_data_df):
  import random

  row, col = entire_data_df.shape
  row_count_for_train = round(row*0.8)   # 80% selected as train data

  train_rows = []   #containing the indexes of rows considered for training
  test_rows = []    #containing the indexes of rows considered for testing 

  while len(train_rows) < row_count_for_train:
    random_num = random.randint(0, row-1)  # generating random number
    if random_num not in train_rows:
      train_rows.append(random_num)

  for i in range(row):
    if i not in train_rows:
      test_rows.append(i)
  random.shuffle(test_rows)  #shuffling because our test_rows was containing value in sorted order    

  x_train_y_train = entire_data_df.iloc[train_rows,:]
  x_test = entire_data_df.iloc[test_rows,:-1]
  y_test = entire_data_df.iloc[test_rows,-1]

  return x_train_y_train , x_test, y_test   # x_train_y_train contains x_train and y_train combined in single dataframe

## DEMO
testing1, testing2, testing3 = split_data(entire_df)
print(testing1.shape)
print(testing2.shape)
print(testing3.shape)

(10367, 9)
(2592, 8)
(2592,)


**ATTRIBUTE SELECTION METHOD**

In [7]:
""" ATTRIBUTE SELECTION METHOD """

## utility function for log base 2
def log(n):
  if n==0:
    return 0
  return math.log2(n)


# calculating info(d) part     gain(attr) = info(d) - info(attr)_d 
def info_of_d(data_df):
  res = 0
  label , freq = np.unique(np.array(data_df.iloc[:,-1]), return_counts = True)  
  frame = dict(zip(label,freq))
  
  for key, val in frame.items():
    res += ((-val/len(data_df)) * (log(val/len(data_df))))
  return res


## this function returns info(attr)_d part for the attribute passed as argument
def calculate_info_attr_d(data_df, attr): 
  temp_df = data_df[[attr, list((data_df.columns))[-1]]]
  cols = ["att", "label"]
  temp_df.columns = cols
  labels_list = np.unique(np.array(temp_df['label']))   # ['no', 'yes']
  total_rows = len(temp_df)                        # 14 initially
  
  catagory , freq = np.unique(np.array(temp_df.iloc[:,0]), return_counts = True)
  catagories_dict = dict(zip(catagory,freq))         # {'middle-aged': 4, 'senior': 5, 'youth': 5}
  
  final_gain = 0
  for this_catagory, occurance in catagories_dict.items():
    log_term_for_this_catagory = 0
    for this_label in labels_list:
      log_term_for_this_catagory += ((-len(temp_df[(temp_df.att == this_catagory) & (temp_df.label == this_label)]) / occurance) * (log(len(temp_df[(temp_df.att == this_catagory) & (temp_df.label == this_label)]) / occurance)))
    final_gain += (occurance/ total_rows) * log_term_for_this_catagory
  return final_gain  


def attribute_selection_method(data_df):
  # Base Condition
  # in case there is no attribute left to make partition  
  if len(list(data_df.columns)) == 1:
    return "attribute finish" 

  attribute_list = list(data_df.iloc[:,:-1].columns)  ## only taken attribute except the label column
  info_d = info_of_d(data_df)
  # if info_d == 0 we can say we achieve a pure partition
  if info_d == 0:
    return "pure partition" 


  max_gain = 0
  for this_attribute in attribute_list:
    info_attr_d = calculate_info_attr_d(data_df, this_attribute)
    gain = info_d - info_attr_d  ## final gain for the features
    # print( this_attribute , gain) #demo print
    if gain > max_gain:
      max_gain = gain    # keep tracking for max gain for any attribute on which we will further split
      partition_attribute = this_attribute
  return partition_attribute  # returns the attrbute on which we will perform splitting


## DEMO
# attribute_selection_method(entire_df)


**FUNCTION TO SPLIT A DATAFRAME WITH RESPECT TO AN ATTRIBUTE**

In [8]:
## split_dataframe function, splits dataset with respect to a features
# as example in first case we are splitting with respect to age

def split_dataframe(data_df, attr):
  attribute_list = list(data_df.columns)   # ["age", "income", "student" , "credit-rating", "buys"]
  temp_df = data_df[attr]
  catagory_list = list(set(temp_df))       # for age ["middle_aged", "youth", "senior"]
  index = attribute_list.index(attr)       # attr= "age"  -> 0   , because "age" present at 0
  
  frame_list_dict = {}
  for this_catagory in catagory_list:
    frame = data_df[(data_df.iloc[:,index] == this_catagory)]
    frame = frame.drop(attr , 1)
    frame_list_dict[this_catagory] = frame
  return frame_list_dict

# DEMO
# output = split_dataframe(entire_df, 'age')
# for key , val in output.items():
#   print(key)
#   print(val)
#   print("\n")

**BUILD THE TREE MODEL**

In [9]:
#### BUILDING THE DECISION TREE ##

class tree_node:
  def __init__(self, data_df):
    self.data = data_df 
    self.split_attribute = None
    self.children = {}
    self.parent = None
    self.label = None
    self.parents_catagory = None

  # returns the level of a particular node in the tree
  def get_level(self):
    level = 0
    p = self.parent
    while p:
      level += 1
      p = p.parent
    return level

  def print_tree(self):
    spaces = ' ' * self.get_level() * 12
    prefix = spaces if self.parent else ""
    if self.split_attribute == None:
      print(prefix + self.parents_catagory + "->"+ self.label)
    elif self.parents_catagory == None:
        print("root --> " + self.split_attribute )  
    else:  
      print(prefix + self.parents_catagory + "->" + self.split_attribute)
    if self.children:
      for key, child in self.children.items():
        child.print_tree()

  def add_child(self, child, catagory):  # child is a instance of treenode
    child.parent = self
    self.children[catagory] = child   ## child is the instance of the treenode
    
###############    END OF TREE_NODE CLASS   #################




# this function finds the maximum occured element from a list
def classify_max_occurance(l):
  max_time_occured =  max(set(l), key = l.count)  
  return max_time_occured

leaf_nodes = []  # containing all the labels of leaf nodes of the tree

def build_tree(self):  
  partition_attribute = attribute_selection_method(self.data)

  """STOPPING CRITERIA"""
  # in case we achieve pure partition assigning label value with the label
  if partition_attribute == "pure partition":
    self.label = self.data.iloc[0,-1]
    leaf_nodes.append(self.label) 
    return

  # in case there is no more attribute to check further, assigning label with the max occurance of labels in dataframe present at that node
  if partition_attribute == "attribute finish":
    max_occurance = classify_max_occurance(list(self.data.iloc[:,-1]))
    self.label = max_occurance
    leaf_nodes.append(self.label) 
    return self
 
  # NOTE :  in case the dataframe becomes empty that case handled in classify function

  frame_list_dict = split_dataframe(self.data, partition_attribute)

  for catagory, this_frame in frame_list_dict.items():
    tmp_node = tree_node(this_frame)
    self.add_child(tmp_node, catagory)
    if tmp_node.parent:
      tmp_node.parent.split_attribute = partition_attribute
      tmp_node.parents_catagory = catagory    
    build_tree(tmp_node)   ## recursively form sub-tree for each children if need



""" Main Function """      
if __name__ == '__main__':
  x_train_y_train , x_test, y_test = split_data(entire_df)

  partition_attribute = attribute_selection_method(x_train_y_train)

  # building tree only on training dataset
  root = tree_node(x_train_y_train)
  root.split_attribute = partition_attribute
  
  build_tree(root)

  # finding label that occurs maximum time in the tree
  classify_res_if_exception = classify_max_occurance([i for i in leaf_nodes if i != 'exception'])
  
  # print the tree
  root.print_tree()    
  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  del sys.path[0]


root --> health
            recommended->has_nurs
                        less_proper->parents
                                    great_pret->social
                                                nonprob->priority
                                                problematic->housing
                                                            convenient->finance
                                                                        convenient->priority
                                                                        inconv->children
                                                                                    2->form
                                                                                                incomplete->spec_prior
                                                                                                complete->priority
                                                                                    1->form
                                   

**PREDICT FROM THE TREE MODEL**

In [10]:

# classify function to classify a single unseen datarow

def classify(self , test_row_dict):
  if self.split_attribute == None:
    return self.label
  elif test_row_dict[self.split_attribute] not in self.children:
    return "exception"   # in case catagory of that feature not present in dictionary
  else:  
    return classify(self.children[test_row_dict[self.split_attribute]] , test_row_dict) 

## this function predicts the label of each test_row from x_test and return y_pred list
def predict_from_tree(x_test):
  col_names = list(x_test.columns)

  y_pred = []
  x_test_array = np.array(x_test) 
  for this_test_row in x_test_array:
    test_row_dict = dict(zip(col_names,this_test_row))
    y_pred.append(classify(root , test_row_dict))

  return y_pred

# calling the function
y_predicted = predict_from_tree(x_test)


# (Exceptional case) Classifying the empty dataframe nodes with the label that occurs maximum time in the tree
for i in range(len(y_predicted)):
  if y_predicted[i] == 'exception':
    y_predicted[i] = classify_res_if_exception

print(y_predicted) # containing all the predicted_labels for each row in x_test


['priority', 'spec_prior', 'not_recom', 'not_recom', 'spec_prior', 'not_recom', 'not_recom', 'not_recom', 'priority', 'priority', 'priority', 'priority', 'not_recom', 'not_recom', 'not_recom', 'not_recom', 'spec_prior', 'priority', 'priority', 'spec_prior', 'spec_prior', 'not_recom', 'priority', 'not_recom', 'spec_prior', 'spec_prior', 'not_recom', 'priority', 'not_recom', 'not_recom', 'spec_prior', 'priority', 'priority', 'not_recom', 'spec_prior', 'not_recom', 'not_recom', 'spec_prior', 'spec_prior', 'very_recom', 'priority', 'not_recom', 'spec_prior', 'spec_prior', 'priority', 'spec_prior', 'priority', 'spec_prior', 'not_recom', 'not_recom', 'priority', 'spec_prior', 'not_recom', 'not_recom', 'not_recom', 'spec_prior', 'priority', 'spec_prior', 'priority', 'priority', 'priority', 'spec_prior', 'spec_prior', 'not_recom', 'not_recom', 'not_recom', 'not_recom', 'not_recom', 'not_recom', 'spec_prior', 'priority', 'not_recom', 'priority', 'priority', 'spec_prior', 'spec_prior', 'priority

**SHOWING RESULTS**

In [11]:
## showing results

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

matrix = confusion_matrix(y_test, y_predicted)

print(f"\nconfusuin matrix :\n {matrix}\n\n")


diagonal_sum = 0
total_sum = 0

#calculating diagonal sum and total sum from confusion matrix
for i in range(matrix.shape[0]):
  for j in range(matrix.shape[1]):
    if i == j:
      diagonal_sum += matrix[i,j]
    total_sum += matrix[i,j]
print(f"\nAccuracy = {(diagonal_sum/total_sum)*100}%\n\n")


print(f"{classification_report(y_test, y_predicted)}" )


confusuin matrix :
 [[865   0   0   0]
 [  0 839   2   7]
 [  0   7 796   0]
 [  0  21   0  55]]



Accuracy = 98.57253086419753%


              precision    recall  f1-score   support

   not_recom       1.00      1.00      1.00       865
    priority       0.97      0.99      0.98       848
  spec_prior       1.00      0.99      0.99       803
  very_recom       0.89      0.72      0.80        76

    accuracy                           0.99      2592
   macro avg       0.96      0.93      0.94      2592
weighted avg       0.99      0.99      0.99      2592

